In [1]:
import greenlet
import gevent
import grequests
from PIL import Image
import pytesseract
import sys
import os
import heapq
import random
import re
import requests
import json
import time
import concurrent.futures

D:\Anaconda\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [8]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
    'Host' : 'dxyq.njust.edu.cn',
    'Proxy-Connection' : 'keep-alive',
    'Referer' : 'http://dxyq.njust.edu.cn/norderday.jsp?equipId=4af7d10b44723d690144726efb330007&time=1543507200000&ro=false',
    'Origin' : 'http://dxyq.njust.edu.cn',
    'Accept' : 'application/json, text/javascript, */*',
    'Accept-Language' : 'zh-CN,zh;q=0.9',
    'Accept-Encoding' : 'gzip, deflate'
}

proxies = {
    'http': 'http://xzproxy.cnsuning.com:8080',
    'https': 'https://xzproxy.cnsuning.com:8080'
}

getImageHeaders = {
    'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Upgrade-Insecure-Requests' : '1',
    'Proxy-Connection' : 'keep-alive',
    'Pragma' : 'no-cache',
    'Accept-Language' : 'zh-CN,zh;q=0.9',
    'Accept-Encoding' : 'gzip, deflate',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
    'Host' : 'dxyq.njust.edu.cn'
}

cookies = {
    'JSESSIONID' : '2B7CDF832E3CE8B1344F89C9392DEE11', #每天需替换一次
    'iPlanetDirectoryPro' : 'e1emg6UdVyafeRWIFvZLCv', #每天需替换一次
    'MOD_AUTH_CAS' : 'MOD_AUTH_ST-245968-AOqyxHYZsIwPrlG9FMMZ1543797844235-bhIs-cas', #每天需替换一次
}

params = {
    'equipId' : '4af7d10b44723d690144726efb330007',
    'startDate' : '2018-12-10 9:00', #每次抢需替换一次
    'endDate' : '2018-12-10 10:00', #每次抢需替换一次
    'content' : 'testtesttesttesttest',
    'noMerge' : 'false',
    'operatorType' : '1',
    'userno' : '',
    'validate' : ''
}

errorMap = {
    '82 79' : '8279',
    '444a' : '4448',
    '43 72' : '4372',
    '93 76' : '9376'
}
 
#二值化,输入阈值和文件地址
def binaryzation(threshold,image_address):
    image=Image.open(image_address)#打开图片
    image=image.convert('L')#灰度化
    table=[]
    for x in range(256):#二值化
        if x<threshold:
            table.append(0)
        else:
            table.append(1)
    image=image.point(table,'1')
    return image

def main():
    while(True):
        if time.strftime("%M", time.localtime(time.time())) == '00' or time.strftime("%M%S", time.localtime(time.time())) == '5957':
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
            submitRequests()
                

def submitRequests():
    image = requests.get('http://dxyq.njust.edu.cn/image', cookies=cookies, headers = getImageHeaders, proxies=proxies)
    validateNum = getImageNumStr(image)
    validateNum = twiceCorrect(validateNum)
    params['validate'] = validateNum
    #print(params)
    ''' 
        这是grequests框架提供的异步提交请求，速度很快，但不适用于此场景，故放弃
        tasks = [grequests.post('http://dxyq.njust.edu.cn/ajax/orderSave.action', cookies=cookies, headers=headers, data=params, proxies=proxies) for i in range(10)]
        allResponse = grequests.map(tasks, size=10)
        for response in allResponse:
            response.encoding = 'utf-8'
            content = json.loads(response.content)
            res = content['result']['result']
            message = content['result']['message']
            print('%s:  申请结果：%s,  信息：%s' % (time.strftime("%H:%M:%S", time.localtime(time.time())), res, message))
    '''
    response = requests.post('http://dxyq.njust.edu.cn/ajax/orderSave.action', cookies=cookies, headers=headers, data=params, proxies=proxies)
    response.encoding = 'utf-8'
    content = json.loads(response.content)
    res = content['result']['result']
    message = content['result']['message']
    print('%s:  申请结果：%s,  信息：%s' % (time.strftime("%H:%M:%S", time.localtime(time.time())), res, message))
        
def twiceCorrect(src):
    return errorMap.get(src, src)
        
def getNextImageCount():
    baseImagePath = 'D:/images'
    imageFileNames = os.listdir(baseImagePath)
    pattern = re.compile('(\d+)\.jpg')
    maxNum = -1
    for imageFile in imageFileNames:
        match = re.findall(pattern, imageFile)
        #print(imageFile)
        num = int(match[0])
        if num > maxNum:
            maxNum = num
    return maxNum + 1
    
def transHundersImages():
    basePath = 'D:/images/%d.jpg'
    errorCount = 0
    errorList = []
    errorRes = []
    for i in range(1, 586):
        imagePath = basePath % (i)
        if not os.path.exists(imagePath):
            continue
        image = binaryzation(141, imagePath)
        result = pytesseract.image_to_string(image)
        result = twiceCorrect(result)
        print('%-5dresult: %s' % (i, result))
        if len(result) != 4 or not result.isdigit():
            errorCount += 1
            image.show()
            errorList.append(i)
            errorRes.append(result)
    print('错误个数为： %d' % (errorCount))
    print('错误的图为：', errorList)
    print('错误的结果为：', errorRes)
    

def getImageNumStr(response):
    image = response.content
    #num =getNextImageCount()
    #path = 'D:/images/%d.jpg' % num
    path = 'D:/images/latest.jpg'
    #print('%s.time:%s' % ('1', time.time()))
    with open(path, 'wb') as f:
        f.write(image)
    #print('%s.time:%s' % ('2', time.time()))
    image=binaryzation(141, path)
    #print('%s.time:%s' % ('3', time.time()))
    #主要耗时是网络和识别图像的过程，识别图像平均耗时300ms，以后重点优化在这里
    result=pytesseract.image_to_string(image)
    #print('%s.time:%s' % ('4', time.time()))
    #print('图片写入D:/images成功，路径为%s, 识别图片验证码为%s' % (path, result))
    return result
    
#main()
#transHundersImages()
#getNextImageCount()

D:\Anaconda\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
main()

08:59:58:  申请结果：False,  信息：预约的结束时间必须距离当前时间168小时内！
08:59:59:  申请结果：False,  信息：验证码错误, 请重新输入！
08:59:59:  申请结果：True,  信息：您已登记预约'FEI Quanta 250F场发射环境扫描电镜'，请等待您的老师和该设备负责人批准！
09:00:00:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:01:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:02:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:03:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:04:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:05:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:06:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-12-10 星期一 09:00)
09:00:06:  申请结果：False,  信息：'FEI Quanta 250F场发射环境扫描电镜'已经被常宜彬预约(2018-12-10 星期一 08:00--2018-1

Traceback (most recent call last):
  File "D:\Anaconda\lib\site-packages\gevent\_ffi\loop.py", line 230, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2018-12-03T01:00:11Z


KeyboardInterrupt: 

D:\Anaconda\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
#'D:/images/%d.jpg' % 3

'D:/images/3.jpg'

In [19]:
#os.listdir('D:/images')

['1.jpg',
 '10.jpg',
 '2.jpg',
 '3.jpg',
 '4.jpg',
 '5.jpg',
 '6.jpg',
 '7.jpg',
 '8.jpg',
 '9.jpg']

In [27]:
#[random.randint(0, 100) for i in range(20)]

[99, 8, 59, 91, 60, 46, 1, 31, 97, 2, 24, 36, 96, 11, 24, 95, 9, 35, 94, 47]

In [119]:
#pattern = 'D:/images/(\d+)\.jpg'
#result = re.findall(pattern, 'D:/images/311.jpg')
#result[0]

"\npattern = 'D:/images/(\\d+)\\.jpg'\nresult = re.findall(pattern, 'D:/images/311.jpg')\nresult[0]\n"

In [69]:
#image = requests.get('http://dxyq.njust.edu.cn/image', cookies=cookies, headers = getImageHeaders, proxies=proxies)
#getImagePath(image)

图片写入D:/images成功，路径为D:/images/11.jpg


'D:/images/11.jpg'

In [67]:
#result = requests.get('http://www.baidu.com', verify = False)
#result.encoding = 'utf-8'
#result.status_code

ConnectionError: HTTPConnectionPool(host='www.baidu.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000007A9F240>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))

In [39]:
#'12f0'.isdigit()

In [44]:
def callback1(response):
    print(response.content)
tasks = []
tasks.append(grequests.get('http://www.baidu.com', proxies = proxies))
tasks.append(grequests.get('http://www.suning.com', proxies = proxies))
responses = grequests.map(tasks, size=3)
print(responses[1].content)

b'<!DOCTYPE html>\r\n<html lang="zh-cn">\r\n<head>\r\n<meta charset="utf-8">\r\n<link rel="shortcut icon" href="//www.suning.com/favicon.ico" type="image/x-icon">\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<meta name="keywords" content="\xe8\x8b\x8f\xe5\xae\x81\xe6\x98\x93\xe8\xb4\xad\xe7\xbd\x91\xe4\xb8\x8a\xe5\x95\x86\xe5\x9f\x8e,\xe8\x8b\x8f\xe5\xae\x81\xe7\x94\xb5\xe5\x99\xa8,Suning,\xe6\x89\x8b\xe6\x9c\xba,\xe7\x94\xb5\xe8\x84\x91,\xe5\x86\xb0\xe7\xae\xb1,\xe6\xb4\x97\xe8\xa1\xa3\xe6\x9c\xba,\xe7\x9b\xb8\xe6\x9c\xba,\xe6\x95\xb0\xe7\xa0\x81,\xe5\xae\xb6\xe5\xb1\x85\xe7\x94\xa8\xe5\x93\x81,\xe9\x9e\x8b\xe5\xb8\xbd,\xe5\x8c\x96\xe5\xa6\x86\xe5\x93\x81,\xe6\xaf\x8d\xe5\xa9\xb4\xe7\x94\xa8\xe5\x93\x81,\xe5\x9b\xbe\xe4\xb9\xa6,\xe9\xa3\x9f\xe5\x93\x81,\xe6\xad\xa3\xe5\x93\x81\xe8\xa1\x8c\xe8\xb4\xa7"/>\r\n<meta name="description" content="\xe8\x8b\x8f\xe5\xae\x81\xe6\x98\x93\xe8\xb4\xad-\xe7\xbb\xbc\xe5\x90\x88\xe7\xbd\x91\xe4\xb8\x8a\xe8\xb4\xad\xe7\x8

D:\Anaconda\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
